## Data analysis and Pre-processing
- Analyze data distribution
    - Categorical Data Distribution
    - Numerical Data Distribution
    - Analyze data (find correlations)
    - Find outliers and tag them
## Hybrid collaborative filtering
- Feature Engineering
- Vehicle similarity Matrix
- Ideal Profile
- Hybrid IBCF

In [ ]:
# IMPORTS
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from mapsAPI import run_route_pipeline

from logger_setup import get_file_logger

logger = get_file_logger("data_pre-processing_")


#====================
#== Configuration ==
#====================
# Quantity of vehicles output
top_n = 20

# Choose between using a custom route or a predefined one 
custom_route = False
if custom_route:
    logger.debug(f"Using custom route")
    '''
    # Example
    run_route_pipeline(
            origin="Girona, Spain",
            destination="Sevilla, Spain",
            waypoints=["Tarragona, Spain", "Valencia, Spain", "Madrid, Spain"]
        )
    '''
    # Write origin, destination and waypoints
    route_data = run_route_pipeline(
        origin="7100 Estremoz, Portugal", 
        destination="7350 Elvas, Portugal",
        waypoints=[] # Waypoints CAN BE EMPTY, must be written as "place 1", "place 2"
    )    

else:
    logger.debug(f"Using predefined route") 
    # Select predefined route data up to 51
    route = 1 # 0..51
    route_df = pd.read_csv('datasets/routes/route_info_data.csv')
    route_data = route_df.iloc[route]
    logger.debug(f"Using route with\n{route_data.head}")

# Fullfill preferences
user_prefs = {
    "max_price_euro": 50000, # Up to 383.318
    "max_car_age": 20, # Up to 25
    "wanted_fuel_types": ["Diesel","Petrol"], #"Diesel", "Petrol", "CNG", "LPG"
    "wanted_transmissions": ["Manual","Automatic"] # "Manual","Automatic"
}

dataset_save_path = "datasets/proper_ibcf_results.csv"
df = pd.read_csv("datasets/vehicles/clean_car_df.csv")
df = df.drop(columns="price_lakh")

# Categorical Data Distribution

In [ ]:
object_columns = df.select_dtypes('object')

# Get numerical columns
numerical_cols = df.select_dtypes(include=np.number).columns.drop(["owner_number","seats"])
# Get categorical columns
catregorical_cols = ["transmission","owner_number","seats","fuel_type","location"]
# Create subplots
fig, axes1 = plt.subplots(2, 3, figsize=(10, 5))
axes1 = axes1.flatten()

# Plot each numerical column with histogram and KDE
for i, col in enumerate(catregorical_cols):
    sns.countplot(data=df,x=col, ax=axes1[i],palette='viridis',hue=col, legend=False)
    axes1[i].set_title(col)
    axes1[i].tick_params(axis='x', rotation=75)  # Rotate x-axis labels for better readability

for i in range(len(catregorical_cols), len(axes1)):
    axes1[i].set_visible(False)

plt.tight_layout()
plt.show()

# Numerical Data Distribution

In [ ]:
fig, axes2 = plt.subplots(3, 3, figsize=(10, 8))
axes2 = axes2.flatten()

# Plot each numerical column with histogram and KDE
for i, col in enumerate(numerical_cols):
    sns.histplot(data=df, x=col, stat='percent',kde=True, ax=axes2[i], bins=30)
    axes2[i].set_title(col)
    axes2[i].tick_params(axis='x', rotation=75)  # Rotate x-axis labels for better readability

# Hide unused subplots
for j in range(len(numerical_cols), len(axes2)):
    axes2[j].set_visible(False)

plt.tight_layout()
plt.show()

## Analyze outliers

Since we are working on a dataset with cars we can't really point out and delete outliers since there are many types of cars with different specifications.

In [ ]:
# Finding outliers using IQR method for numerical columns
def find_outliers_iqr(df):
    outlier_index = dict()
    info = []
    numeric_columns = df.select_dtypes(include=np.number).drop(columns=["owner_number","seats"])
    for col in numeric_columns:
        Q1 = df[col].quantile(0.25) # Quantile lower
        Q3 = df[col].quantile(0.75) # Quantile upper
        IQR = Q3 - Q1 # Interquartile range
        lower_quantile = Q1 - (1.5 * IQR) # Lower bound
        upper_quantile = Q3 + (1.5 * IQR) # Upper bound

        outliers = df[(df[col]<lower_quantile) | (df[col]> upper_quantile)] # Get data from below lower bound and above upper bound
        outlier_index[col] = outliers.index.tolist() # Get the index of the outliers

        info.append({
            'feature': col,              
            'Q1': Q1,
            'Q3': Q3,
            'IQR': IQR,
            'Lower Bound': lower_quantile,
            'Upper Bound': upper_quantile        
        })
    return pd.DataFrame(info), outlier_index

def plot_outliers(df, outlier_index):
    # Create a single figure with 7 rows and 2 columns
    fig, axes = plt.subplots(7, 2, figsize=(15, 25))
    
    # Get the list of columns with outliers
    columns = list(outlier_index.keys())
    
    for i, (col, indices) in enumerate(outlier_index.items()):
        # First column - Boxplot
        sns.boxplot(x=df[col], ax=axes[i, 0])
        axes[i, 0].set_title(f'Boxplot de {col}', fontsize=14)
        axes[i, 0].set_xlabel(col)
        
        # Second column - Scatterplot with outliers
        # Paint all the points in the scatterplot
        sns.scatterplot(x=df.index, y=df[col], label='Datos', alpha=0.6, ax=axes[i, 1])
        # Paint the outliers in red over the scatterplot from before
        sns.scatterplot(x=indices, y=df.loc[indices, col], color='red', label='Outliers', s=60, ax=axes[i, 1])
        axes[i, 1].set_title(f'Posibles outliers en {col}', fontsize=14)
        axes[i, 1].set_xlabel('Indice')
        axes[i, 1].set_ylabel(col)
        #axes[i, 1].legend()
        axes[i, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left',markerfirst=False)
    
    # Hide any unused subplots if there are fewer than 7 variables
    for j in range(len(columns), 7):
        axes[j, 0].set_visible(False)
        axes[j, 1].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Call the function
summary, outlier_index = find_outliers_iqr(df)
print(summary)
plot_outliers(df, outlier_index)

In [ ]:
def create_quartile_tags(df, outlier_index, column_mappings=None):
    """
    Creates binary columns for quartile-based outlier tagging
    
    Parameters:
    df: DataFrame with the data
    outlier_index: Dictionary with column names as keys and outlier indices as values
    column_mappings: Dictionary mapping column names to their low/high labels
                    Example: {'age': ('Old_car', 'New_car'), 'price': ('Cheap_car', 'Expensive_car')}
    
    Returns:
    DataFrame with new binary columns added
    """
    
    # Default mappings if none provided
    if column_mappings is None:
        column_mappings = {}
    
    # Create a copy of the dataframe to avoid modifying the original
    df_tagged = df.copy()
    
    for col in outlier_index.keys():
        # Calculate quartiles
        Q1 = df[col].quantile(0.25) # Quantile lower
        Q3 = df[col].quantile(0.75) # Quantile upper
        IQR = Q3 - Q1 # Interquartile range
        q25 = Q1 - (1.5 * IQR) # Lower bound
        q75 = Q3 + (1.5 * IQR) # Upper bound

        #q25 = df[col].quantile(0.25)
        #q75 = df[col].quantile(0.75)
        
        # Get custom labels or create default ones
        if col in column_mappings:
            low_label, high_label = column_mappings[col]
        else:
            # Create default labels based on column name
            low_label = f"low_{col}"
            high_label = f"high_{col}"
        
        # Create binary columns
        # 1 for values below Q25, 0 otherwise
        df_tagged[low_label] = (df[col] < q25).astype(int)
        
        # 1 for values above Q75, 0 otherwise  
        df_tagged[high_label] = (df[col] > q75).astype(int)
        
        print(f"Created columns for {col}:")
        print(f"  - {low_label}: {df_tagged[low_label].sum()} records tagged")
        print(f"  - {high_label}: {df_tagged[high_label].sum()} records tagged")
        print()
    
    return df_tagged

# Example usage:
# Define custom mappings for your columns
column_mappings = {
    'age': ('Old_car', 'New_car'),
    'price': ('Cheap_car', 'Expensive_car'),
    'mileage': ('Low_mileage', 'High_mileage'),
    'engine_size': ('Small_engine', 'Large_engine')
    # Add more mappings as needed for your specific columns
}

# Create the tagged dataframe
df_tagged = create_quartile_tags(df, outlier_index, column_mappings)

# Display the new columns
print("New binary columns created:")
new_columns = [col for col in df_tagged.columns if col not in df.columns]
print(new_columns)

# Show some examples
print("\nSample of tagged data:")
#print(df_tagged[new_columns].head(10))
df_tagged.head()

# Hybrid IBCF

## Correlation matrix between both vehicles and route features

As we can see vehicle features does not correlate with route features so we'll have to do feature engineering to make the features meaningful for the cosine similarity

In [ ]:
route_df = pd.read_csv('datasets/routes/route_info_data.csv')
vehicle_num_df = df.select_dtypes(include=np.number)
route_num_df = route_df.select_dtypes(include=np.number)

# OPTIONAL: Repeat route_df to match number of vehicle rows (for correlation)
# For meaningful correlation, both DataFrames should have same number of rows
route_repeated = pd.concat([route_num_df]*len(df), ignore_index=True)

# Combine them
combined_df = pd.concat([vehicle_num_df.reset_index(drop=True), route_repeated], axis=1)

# Now compute correlation
correlation_matrix = combined_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de correlacion de las variables numericas de vehiculos y rutas')
plt.tight_layout()
plt.show()


## Feature engineering 
Each car specification (like age, horsepower, engine size, price) originally has a totally different range.
To make comparisons fair, we scale all of them between 0 and 1 using normalization.
Then we create three combined scores that better reflect what really matters in a vehicle:
- Performance = mostly power + some engine size
- Economy = mostly fuel efficiency + some affordability
- Overall quality = a mix of reliability, performance, and economy
## Vehicle Similarity Matrix
Then we compare every car to every other car using cosine similarity, which basically checks how "aligned" two cars are across all features. This gives us a score for each pair of cars: how similar their capabilities are.
## Ideal vehicle profile
We create a made-up car profile that would be perfect for the route, for example, a steep long trip would call for high power, fuel efficiency, and reliability.
We then compare that ideal car to all real cars using cosine similarity again, giving each real car a direct match score.
## Hybrid IBCF
For each real car A, we check how similar it is to every other car (based on the earlier car-to-car matrix), Then weight those similarites by how well those other cars match the ideal profile and this gives us an IBCF score.
## Hybrid vehicles ranking
Then we normalize both scores (ideal similarity and ibcf score) and average them 50/50 to get a hybrid score, which balances how well a car matches the route and how similar it is to cars that do


In [ ]:
def hybrid_ibcf(route_data,vehicle_df,top_n_vehicles):
    #=========================
    #== Feature Engineering ==
    #=========================

    scaler = MinMaxScaler()
    vehicle_features = vehicle_df.copy()

    logger.debug("Creating new feature vectors")
    # Create new feature vectors to enrich dataset information
    score_map = {
        'reliability_score': (vehicle_features[['car_age']].max() - vehicle_features[['car_age']]).values,
        'fuel_efficiency_score': vehicle_features[['mileage_kmpl']].values,
        'power_score': vehicle_features[['power_bhp']].values,
        'size_score': vehicle_features[['engine_cc']].values,
        'affordability_score': (vehicle_features[['price_euro']].max() - vehicle_features[['price_euro']]).values
    }

    logger.debug(f"Normalizing newly created features")
    # Normalize the values for each feature
    for key, value in score_map.items():
        vehicle_features[key] = scaler.fit_transform(value).flatten()
    
    logger.debug(f"Setting up weights:")
    # Create composite features to make cosine-similarity and IBCF more meaningful
    weight = {
        #Performance score
        'power_score': 0.6,
        'size_score': 0.4,
        #Economy score
        'fuel_efficiency_score':0.7,
        'affordability_score':0.3,
        #Overall quality score
        'reliability_score':0.4,
        'performance_score':0.3,
        'economy_score':0.3
    }
    logger.debug(f"Creating new composite features")
    # performance_score = (w1) * power_score + (w1) * size_score
    vehicle_features['performance_score'] = (
        weight['power_score'] * vehicle_features['power_score'] + 
        weight['size_score'] * vehicle_features['size_score']
    )
    # economy_score = ((w1) * fuel_efficiency_score) + ((w2) * affordability_score)
    vehicle_features['economy_score'] = (
        weight['fuel_efficiency_score'] * vehicle_features['fuel_efficiency_score']
        +weight['affordability_score']* vehicle_features['affordability_score']
    )
    # overall_quality_score = ((w1) * reliability_score) + ((w2) * performance_score) + ((w3) * economy_score)
    vehicle_features['overall_quality_score'] = (
        weight['reliability_score'] * vehicle_features['reliability_score']
        + weight['performance_score'] * vehicle_features['performance_score']
        + weight['economy_score'] * vehicle_features['economy_score']
    )
    # Group all new features
    unified_features = [
        'reliability_score', 'fuel_efficiency_score', 'power_score', 
        'size_score', 'affordability_score', 'performance_score', 
        'economy_score', 'overall_quality_score'
    ]

    #===============================
    #== Vehicle Similarity Matrix ==
    #===============================
    logger.debug(f"Computing newly created features into a cosine similarity")
    vehicle_matrix = vehicle_features[unified_features].values
    vehicle_similarity_matrix = cosine_similarity(vehicle_matrix)
    np.fill_diagonal(vehicle_similarity_matrix, 0) # Avoid self-similarity
    logger.debug(f"New vehicle matrix generated")
    #===========================
    #== Ideal Vehicle Profile ==
    #===========================
    logger.debug(f"Extracting route features")
    # Route features
    distance_km = route_data['distance_km']
    elevation_gain = route_data['elevation_gain_m']
    avg_slope = route_data['avg_slope']

    # Rule based preference
    # Each metric decides how important the feature is for the ideal vehicle profile
    # Example:
    # Fuel_efficiency_score: The higher the distance the more importance it has over the ideal profile
    #
    logger.debug(f"Creating new ideal vehicle profile based on route features")
    ideal_vehicle = np.zeros(len(unified_features)) # List filled with zeroes
    #reliability_score
    ideal_vehicle[0] = 0.9 if distance_km > 200 else 0.7 if distance_km > 100 else 0.5 
    #fuel_efficiency_score
    ideal_vehicle[1] = 0.8 if distance_km > 150 else 0.6
    #power_score
    ideal_vehicle[2] = 0.9 if elevation_gain > 500 or avg_slope > 4 else 0.7 if elevation_gain > 200 or avg_slope > 2 else 0.5
    #size_score
    ideal_vehicle[3] = ideal_vehicle[2] * 0.8
    #affordability_score
    ideal_vehicle[4] = 0.4 if distance_km > 200 else 0.7
    #performance_score
    ideal_vehicle[5] = 0.6 * ideal_vehicle[2] + 0.4 * ideal_vehicle[3]
    #economy_score
    ideal_vehicle[6] = 0.7 * ideal_vehicle[1] + 0.3 * ideal_vehicle[4]
    #overall_quality_score
    ideal_vehicle[7] = 0.4 * ideal_vehicle[0] + 0.3 * ideal_vehicle[5] + 0.3 * ideal_vehicle[6]
    ideal_profile_weights = pd.DataFrame(ideal_vehicle.reshape(1, -1), columns=unified_features)
    ideal_profile_weights.T.rename(columns={0: 'Ideal Profile values'})

    #=================
    #== Hybrid IBCF == 
    #=================
    logger.debug(f"Computing cosine similarity with new ideal vehicle profile and previous vehicle matrix")
    ideal_similarity = cosine_similarity([ideal_vehicle], vehicle_matrix)[0]
    logger.debug(f"Computing similar ibcf technique to improve vehicle relations")
    ibcf_scores_list = []
    for vehicle_index in range(len(vehicle_features)):
        # similarity of vehicle idx to all other vehicles
        sim_to_others = vehicle_similarity_matrix[vehicle_index]
        # weighted sum: how much these other vehicles match the ideal
        weighted_match = np.sum(sim_to_others * ideal_similarity)
        ibcf_scores_list.append(weighted_match)
    # convert to numpy array for consistency
    ibcf_scores = np.array(ibcf_scores_list)

    #======================
    #== Ranking Vehicles ==
    #======================
    logger.debug(f"Normalizing ideal_similarity matrix and ibcf matrix ")
    # Normalization of ideal and ibcf similarity scores
    normalization_ideal = (ideal_similarity - ideal_similarity.min()) / (ideal_similarity.max() - ideal_similarity.min())
    normalization_ibcf = (ibcf_scores - ibcf_scores.min()) / (ibcf_scores.max() - ibcf_scores.min())
    
    # Alpha: How much you trust the ideal similarity match
    # Beta: How much you trust the IBCF similarity
    
    alpha, beta = 0.5, 0.5
    #logger(f"Defining final hybrid score with {alpha} ideal_vehicle weight and {beta} of ibcf weight")
    hybrid_score = alpha * normalization_ideal + beta * normalization_ibcf

    vehicle_features['hybrid_score'] = hybrid_score
    vehicle_features['direct_similarity'] = ideal_similarity
    vehicle_features['ibcf_score'] = ibcf_scores
    # Could sort values between hybrid, direct and ibcf score
    score_type = 'hybrid_score'
    logger.debug(f"Ranking final vehicles with {score_type}")
    ibcf_results = vehicle_features.sort_values(score_type, ascending=False).head(top_n_vehicles)
    ibcf_results.to_csv(dataset_save_path, index=False)
    logger.info(f"ibcf_results saved to {dataset_save_path}")
    logger.debug(f"Finalizing vehicle ranking")
    return ideal_similarity,ibcf_scores,vehicle_features,ibcf_results,ideal_profile_weights,unified_features

vehicle_df = df_tagged.copy()
# Applying the filters to the dataset
logger.debug(f"Before applying filter: {vehicle_df.shape}")
vehicle_df = vehicle_df[
    (vehicle_df["price_euro"] <= user_prefs["max_price_euro"]) &
    (vehicle_df["car_age"] <= user_prefs["max_car_age"]) &
    (vehicle_df["fuel_type"].isin(user_prefs["wanted_fuel_types"])) &
    (vehicle_df["transmission"].isin(user_prefs["wanted_transmissions"]))
].reset_index(drop=True)
logger.debug(f"After applying filter: {vehicle_df.shape}")

if vehicle_df.empty:
    logger.warning("No vehicles match the user's preferences.")    
else: 
    normalization_ideal,normalization_ibcf,ranked_vehicles_df,top_n_ranked_vehicles_df,ideal_profile,ideal_profile_features = hybrid_ibcf(route_data,vehicle_df,top_n)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 4))
sns.histplot(ranked_vehicles_df['hybrid_score'], bins=50, kde=True, color='blue')
plt.title('Vehicle similarity to the ideal profile')
plt.xlabel('Hybrid score from ideal profile cosine similarity and IBCF')
plt.ylabel('Quantity of vehicles')
plt.grid(True)
plt.show()

plt.figure(figsize=(12,4))
sns.barplot(ideal_profile)
plt.xticks(rotation=45)
plt.title("Feature weight value on ideal_similarity score")
plt.show()

top_n_ranked_vehicles_df.head(10)